In [7]:
import sys, os
sys.path.insert(0, '../')
sys.path.insert(0, '../python_src/')

import numpy as np
import scipy as sp

import homology
import chomology

In [8]:

NV = 6
NE = 5

DIM = 2

edgei = [0, 0, 0, 0, 0]
edgej = [1, 2, 3, 4, 5]

L = np.array([10.0, 10.0], float)

comp = chomology.construct_graph_complex(NV, NE, edgei, edgej, False, False)

for i in range(comp.ncells):
    print(i, comp.get_dim(i), comp.get_facets(i), comp.get_cofacets(i))


vert_pos = np.zeros(DIM*NV, float)
vert_pos[0:DIM] = [0, 0]
vert_pos[DIM:2*DIM] = [-1, 0]
vert_pos[2*DIM:3*DIM] = [1, 0]
vert_pos[3*DIM:4*DIM] = [0, -1]
vert_pos[4*DIM:5*DIM] = [0, 1]
vert_pos[5*DIM:6*DIM] = [1, 1]

print("pos", vert_pos)
    
corners = chomology.find_corners(comp, DIM, NV, vert_pos, L)

print(corners)

0 0 [] [6, 7, 8, 9, 10]
1 0 [] [6]
2 0 [] [7]
3 0 [] [8]
4 0 [] [9]
5 0 [] [10]
6 1 [0, 1] []
7 1 [0, 2] []
8 1 [0, 3] []
9 1 [0, 4] []
10 1 [0, 5] []
pos [ 0.  0. -1.  0.  1.  0.  0. -1.  0.  1.  1.  1.]
[[[3, 1], [4, 1], [3, 2], [5, 2], [5, 4]], [], [], [], [], []]
